本篇的目的是介绍蒙特卡洛模拟法的步骤，主要是如何使用蒙特卡洛法获取衍生品，尤其是期权的价格

考虑在第一篇中讲过的几何布朗运动：
$$
dS = bSdt+\sigma S dz \tag{4.1}
$$
若设$Y = lnS$,根据伊藤引理，有：
$$
dY = \frac{\partial Y}{\partial S} dS + \frac{1}{2}\frac{\partial^2 Y}{\partial S^2} (dS)^2 \tag{4.2}
$$

计算偏导并将偏导和(4.1)带入，可得：
$$
dY = (b-\frac{1}{2}\sigma^2)dt + \sigma dz \tag{4.3}
$$

故而积分可得：
$$
Y_t - Y_0 = (b-\frac{1}{2}\sigma^2)t + \sigma (z_t-z_0) \tag{4.4}
$$

因为$z_0 =0,Y_0=lnS_0$,所以有：
$$
Y_t =lnS_0 +  (b-\frac{1}{2}\sigma^2)t + \sigma z_t \tag{4.5}
$$
其$z_t = \epsilon\sqrt{t}(\epsilon \sim N(0,1))$为标准正态分布的随机变量，所以最后解得：
$$
S_t = S_0 e^{(b-\frac{1}{2}\sigma^2)t + \sigma \epsilon\sqrt{t}} \tag{4.6}
$$


根据(4.6)即可模拟出一段时间内路径内的衍生品价格。

（1） 模拟欧式期权

In [20]:
import numpy as np
import matplotlib.pyplot as plt
import numba
import scipy.stats as stats
import pandas as pd
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [2]:
def geo_brownian(steps,paths,T,S0,b,sigma):
    #模拟路劲生成
    dt = T/steps #时间间隔dt
    S_path = np.zeros((steps+1,paths)) #初始化路径矩阵,，用来准备储存模拟情况
    S_path[0] = S0 #起点设置
    for step in range(1,steps+1):
        rn = np.random.standard_normal(paths) #创造随机数
        S_path[step] = S_path[step-1] * np.exp((b-0.5*sigma**2)*dt + sigma*np.sqrt(dt)*rn)
    return S_path
    

In [3]:
def MC_Euro_Option(steps,paths,T,S0,X,sigma,b,r):
    #模拟欧式期权价格
    S_path = geo_brownian(steps,paths,T,S0,b,sigma) #模拟路径
    value = np.exp(-r*T)*np.maximum(S_path[-1]-X,0).mean()
    return value

In [4]:
C = MC_Euro_Option(steps=250,paths=5000000,T=1,S0=100,X=99,sigma=0.2,b=0.03,r=0.03)
print(C)

9.926707877251987


这一结果和BSM算出来的解析解十分接近

（2）模拟亚式期权

亚式期权的到期回报取决于路劲上资产价格的均值（算术平均或几何平均），这很难得到解析解，因此需要使用蒙特卡洛模拟

In [9]:
def asian_option(steps,paths,T,S0,X,sigma,r,b):
    S_path = geo_brownian(steps,paths,T,S0,b,sigma) #生成路径
    S_mean = S_path.mean(axis=0) #计算每一条路劲上的均价
    value = np.exp(-r*T)*np.maximum(S_mean-X,0).mean() #计算到期回报的均值
    return value


In [10]:
asian_option(steps=250,paths=500000,T=1,S0=100,X=99,sigma=0.2,r=0.03,b=0.03)

5.8046783320159685

（3）蒙卡劣势

模拟1000条路径相较于模拟100条路径得到的标准差肯定是更低的，但这种对于方差的降低效果并不高。若模拟N条路径，每条模拟路径的值为$X_i$，标准差为$\sigma$，那么路径均值为$X=\sum X_i/n$，其标准误为$\sigma/\sqrt{n}$。因此路径的提升对于降低模拟标准差的效果会有一个平方根的影响。

因此，提出以下改进方法：

In [ ]:
'''
对偶变量法
'''
def anti_MC(steps,paths,T,S0,K,sigma,r,b):
    dt = T / steps # 求出dt
    S_path_1 = np.zeros((steps+1,paths))   #创建一个矩阵，用来准备储存模拟情况
    S_path_2 = np.zeros((steps+1,paths))   #创建对偶矩阵，用来准备储存模拟情况
    S_path_1[0],S_path_2[0]= S0,S0  #起点设置
    for step in range(1,steps+1):
        rn = np.random.standard_normal(paths) #创造随机数
        S_path_1[step] = S_path_1[step - 1] * np.exp((b-0.5*sigma**2)*dt +sigma*np.sqrt(dt)*rn)
        S_path_2[step] = S_path_2[step - 1] * np.exp((b-0.5*sigma**2)*dt +sigma*np.sqrt(dt)*(-rn))#差异在-rn
    #到期时期权价值，先计算这一步后面再求均值的目的是确保每条路径之间是独立的
    V_end_array = (np.maximum(S_path_1[-1]-K,0)+ np.maximum(S_path_2[-1]-K,0))/2  
    value = np.exp(-r*T)*V_end_array.mean()
    return value
C = anti_MC(steps = 250,paths = 250000,T = 1,S0 = 100,K=99,sigma = 0.2,r=0.03,b=0.03)
print(C)
'''
比直接模拟500000要快
'''

9.91294459863989


（4）模拟美式期权

普通蒙卡无法定价美式期权，美式期权需要在各个steps比较行权价值价值和继续持有的价值，这就需要在没一个视点重新进行一次蒙卡用以计算当前欧式期权价格和立马行权价格，从而对比得到是否继续持有，这种在节点上在模拟新的节点的方式会让蒙卡陷入巨大的循环中。

可以采用最小二乘蒙特卡洛模拟的方法：先通过蒙特卡洛模拟得到现货的价格路径，接着对于现货价格和未来现金流进行OLS，得到基于标的价格的继续持有的条件期望价值，通过比较继续持有和内在价值大小，得到提前行权的时间点，实现对美式期权的定价。


对美式看跌期权估值，行权价为1.1，标的价格为1，无风险利率为6%，模拟的价格路径如下。

In [ ]:
S_path = pd.DataFrame(data={"Path":np.arange(1,9),"t=0":np.ones(8),\
                     "t=1":np.array([1.09,1.16,1.22,0.93,1.11,0.76,0.92,0.88]),\
                     "t=2":np.array([1+.08,1.26,1.07,0.97,1.56,0.77,0.84,1.22]),\
                     "t=3":np.array([1.34,1.54,1.03,0.92,1.52,0.90,1.01,1.34])})
S_path.set_index("Path",inplace =True)
S_path

,t=0,t=1,t=2,t=3
Path,,,,
1,1.0,1.09,1.08,1.34
2,1.0,1.16,1.26,1.54
3,1.0,1.22,1.07,1.03
4,1.0,0.93,0.97,0.92
5,1.0,1.11,1.56,1.52
6,1.0,0.76,0.77,0.90
7,1.0,0.92,0.84,1.01
8,1.0,0.88,1.22,1.34


In [22]:
#计算内在价值
exercise_value = np.maximum(1.1-S_path,0)
exercise_value


,t=0,t=1,t=2,t=3
Path,,,,
1,0.1,0.01,0.02,0.00
2,0.1,0.00,0.00,0.00
3,0.1,0.00,0.03,0.07
4,0.1,0.17,0.13,0.18
5,0.1,0.00,0.00,0.00
6,0.1,0.34,0.33,0.20
7,0.1,0.18,0.26,0.09
8,0.1,0.22,0.00,0.00


In [26]:
itm_index = (exercise_value["t=2"]>0)
X = S_path.loc[itm_index,"t=2"]
Y = exercise_value.loc[itm_index,"t=3"]*np.exp(-0.06)
reg_data = pd.DataFrame(data={"X=股票价格":X,"Y=未来现金流折现":Y})
reg_data

,X=股票价格,Y=未来现金流折现
Path,,
1,1.08,0.000000
3,1.07,0.065924
4,0.97,0.169518
6,0.77,0.188353
7,0.84,0.084759


In [36]:
'''
利用XY做回归，计算继续持有价值holding_value
'''
reg = np.polyfit(x=X,y=Y,deg=2)
print(f"回归方程\n{reg[0]:.3f}X^2+{reg[1]:.3f}X {reg[2]:.3f}")# 回归的系数


回归方程
-1.814X^2+2.983X -1.070


In [ ]:
reg_data['holding_value'] = np.polyval(reg,S_path.iloc[itm_index,'t=2'])
reg_data

In [ ]:
reg = np.polyfit(x = X,y = Y,deg = 2)  #XY做2次最小二乘法
print(f"回归方程\n{reg[0]:.3f}X^2+{reg[1]:.3f}X {reg[2]:.3f}")# 回归的系数
reg_data["holding_value"] = np.polyval(reg,S_path.loc[itm_index,"t=2"])  #求出期望的继续持有价值